In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('heart_failure_clinical_records_dataset.csv') # Veri setinin okunması

In [3]:
df.head() # Veri setinin ilk 5 verisi

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [4]:
df.info() # Veri seti hakkında genel bilgiler. 13 adet sütun 299 adet satır bulunmaktadır ve hiç boş veri yoktur.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [5]:
X=df.drop(["DEATH_EVENT"],axis=1) # İlk 12 sütun bağımlı değişken olarak seçilir.
y=df["DEATH_EVENT"] # Sonuncu sütun hedef değişken olarak seçilir.

In [6]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X) # Makine öğrenmesinin gerçekleştirilmesi için eğitim verilerinin standartizasyonu yapılmıştır.

In [7]:
from sklearn.model_selection import train_test_split # Eğitim-Test bölünmesi işlemi.
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1,test_size=0.3)

In [8]:
from keras.models import Sequential
from keras.layers import Dense

In [9]:
def ANN(optimizer, init_mode): # ANN modeli için metot oluşturulmuştur.
    classifier = Sequential() # ANN modelinin tanımlanması.
    classifier.add(Dense(activation = "relu", input_dim = 12, units = 8, kernel_initializer = init_mode)) # Modele relu fonksiyonunun tanımlanması 
    classifier.add(Dense(activation = "sigmoid", units = 1, kernel_initializer = init_mode)) # Modelin nöron biçimi için Sigmoid fonksiyonunun tanımlanması
    classifier.compile(optimizer = optimizer , loss = 'binary_crossentropy', metrics = ['accuracy']) # Modelin hangi parametrelerle çalışacağının belirlenmesi
    return classifier # Modelin geri gönderilmesi

In [10]:
from sklearn.metrics import confusion_matrix

In [11]:
# Her parametre için bir boş liste tanımlanmıştır.
accuracies = []
precisions = []
recalls = []
f1_scores = []
epochs = []
batch_sizes = []
optimizers = []
init_modes = []
for epoch in [10, 50, 100]: # Epoch için döngünün tanımlanması.
    for batch_size in [10, 20, 40, 60, 80, 100]: # Parti boyutu için döngünün tanımlanması.
        for optimizer in ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']: # Optimizasyon modeli için döngünün tanımlanması
            for init_mode in ['uniform', 'normal', 'zero']: # Başlangıç ağırlıkları için kullanılan modeller için döngünün tanımlanması.
                classifier = ANN(optimizer, init_mode) # ANN sınıflandırıcısı oluşturulur.
                classifier.fit(X_train , y_train, epochs = epoch, verbose=0, batch_size = batch_size) # Sınıflandırıcı eğitilir.
                y_pred = classifier.predict(X_test) # Test verileri ile tahminler oluşturulur.
                y_pred = (y_pred > 0.5) # Pozitif etiketli tahmin verileri çekilir.
                cm = confusion_matrix(y_test,y_pred) # Karmaşıklık matrisi oluşturulur.
                TP, FP, FN, TN = cm[0][0], cm[0][1], cm[1][0], cm[1][1] # TP, FP, FN, TN değerleri alınır.
                accuracies.append((TP+TN)/(TP+TN+FP+FN)) # Doğruluk skoru hesaplanır.
                precisions.append(TP/(TP+FP)) # Precision değeri hesaplanır.
                recalls.append(TP/(TP+FN)) # Recall değeri hesaplanır.
                f1_scores.append(2*((TP/(TP+FP)*TP/(TP+FN))/(TP/(TP+FP)+TP/(TP+FN)))) # F1-Skoru hesaplanır.
                # Çalıştırılan modelin ilgili hiper parametreleri listelere eklenir.
                epochs.append(epoch) 
                batch_sizes.append(batch_size)
                optimizers.append(optimizer)
                init_modes.append(init_mode)

C:\Users\OUZHAN~1\AppData\Local\Temp/ipykernel_4936/792258700.py:22: RuntimeWarning: invalid value encountered in double_scalars
  f1_scores.append(2*((TP/(TP+FP)*TP/(TP+FN))/(TP/(TP+FP)+TP/(TP+FN))))


In [12]:
# Listeler kullanılarak tüm sonuçlar birleştirilerek bir dataframe oluşturulur.
d = {'Optimizer': optimizers, 'Epoch': epochs, 'Batch Size': batch_sizes, 'Accuracy': accuracies, 'Precision': precisions, 'Recall': recalls, 'F1-Score': f1_scores}
results = pd.DataFrame(data=d)

In [13]:
results

,Optimizer,Epoch,Batch Size,Accuracy,Precision,Recall,F1-Score
0,SGD,10,10,0.711111,1.000000,0.711111,0.831169
1,SGD,10,10,0.711111,1.000000,0.711111,0.831169
2,SGD,10,10,0.711111,1.000000,0.711111,0.831169
3,RMSprop,10,10,0.777778,0.953125,0.782051,0.859155
4,RMSprop,10,10,0.777778,0.953125,0.782051,0.859155
...,...,...,...,...,...,...,...
373,Adamax,100,100,0.777778,0.968750,0.775000,0.861111
374,Adamax,100,100,0.711111,1.000000,0.711111,0.831169
375,Nadam,100,100,0.855556,0.921875,0.880597,0.900763
376,Nadam,100,100,0.833333,0.921875,0.855072,0.887218


In [14]:
results[results['Accuracy']==results['Accuracy'].max()] # Doğruluk skoru en yüksek olan sonuçlar gösterilir.

,Optimizer,Epoch,Batch Size,Accuracy,Precision,Recall,F1-Score
141,Adamax,50,10,0.855556,0.921875,0.880597,0.900763
171,RMSprop,50,40,0.855556,0.937500,0.869565,0.902256
180,Adam,50,40,0.855556,0.906250,0.892308,0.899225
229,Nadam,50,80,0.855556,0.906250,0.892308,0.899225
319,RMSprop,100,60,0.855556,0.937500,0.869565,0.902256
361,RMSprop,100,100,0.855556,0.921875,0.880597,0.900763
375,Nadam,100,100,0.855556,0.921875,0.880597,0.900763
